In [1]:
import sys
import os

# Get the root directory
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the src directory to sys.path
src_dir = os.path.join(root_dir, "src")
if src_dir not in sys.path:
    sys.path.append(src_dir)

In [2]:
from dataloader import DataLoader
from trainer import WeightedTrainer, compute_metrics
from utils import *

from typing import List
import yaml

import optuna
from optuna import Trial

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    DataCollatorWithPadding,
)

from peft import (
    LoraConfig,
    TaskType,
    get_peft_model
)


/home/bohan-jiang/start/232-stat-nlp/232-cdecr/.sft/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
config_file = "../batch_config.yaml"
configs = load_config(config_file)

In [4]:
tokenizer = AutoTokenizer.from_pretrained(configs['checkpoint'])
if configs['decoder_only']:
    tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenizer.pad_token = tokenizer.eos_token


data_collator = DataCollatorWithPadding(tokenizer)
dataset = DataLoader().load('../data/', tokenizer)


Map:   0%|                                          | 0/42953 [00:00<?, ? examples/s]


Map:  16%|████▌                       | 7000/42953 [00:00<00:00, 46380.24 examples/s]


Map:  40%|██████████▋                | 17000/42953 [00:00<00:00, 71510.80 examples/s]


Map:  65%|█████████████████▌         | 28000/42953 [00:00<00:00, 82474.58 examples/s]


Map:  95%|█████████████████████████▊ | 41000/42953 [00:00<00:00, 94864.29 examples/s]

...


Map:   0%|                                          | 0/36438 [00:00<?, ? examples/s]


Map:  33%|████████▌                 | 12000/36438 [00:00<00:00, 107777.93 examples/s]


Map:  69%|██████████████████▌        | 25000/36438 [00:00<00:00, 91184.71 examples/s]


Map:  96%|█████████████████████████▉ | 35000/36438 [00:00<00:00, 83002.60 examples/s]


Map: 100%|███████████████████████████| 36438/36438 [00:00<00:00, 85511.74 examples/s]

In [5]:
def objective(trail: Trial, config: Dict):

    lr = trail.suggest_categorical("learning_rate", config['lr'])
    batch_size = trail.suggest_categorical("batch_size", config['batch_size'])

    out_dir = f"../{config['cache_dir']}/{config['model_name']}/lr_{trail.params['learning_rate']}_bsz_{trail.params['batch_size']}"

    train_args = TrainingArguments(
        output_dir=out_dir,
        eval_strategy='epoch',
        save_strategy='epoch',
        learning_rate=lr,
        weight_decay=0.001,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=config['epochs'],
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model='eval_F1',
        greater_is_better=True
    )

    peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        inference_mode=False,
        r=3,
        lora_alpha=16,
        lora_dropout=0.1
    )

    model = AutoModelForSequenceClassification.from_pretrained(config['checkpoint'])
    model = get_peft_model(model, peft_config)
    if config['decoder_only']:
        model.config.pad_token_id = model.config.eos_token_id

    trainer = WeightedTrainer(
        model=model,
        args=train_args,
        train_dataset=dataset['train'],
        eval_dataset=dataset['dev'],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    trainer.train()

    trainer.save_model(out_dir)

    eval_result = trainer.evaluate()

    return eval_result['eval_loss']

In [6]:
study = optuna.create_study(direction="minimize")
study.optimize(lambda trial: objective(trial, configs), n_trials=16)

print("Best hyperparameters:", study.best_params)

[I 2025-02-22 00:26:37,967] A new study created in memory with name: no-name-8cfbb771-b084-4c77-847b-4420d82aea22


Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-1.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/home/bohan-jiang/start/232-stat-nlp/232-cdecr/src/trainer.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Acc,Precision,F1
1,0.102500,0.192814,{'accuracy': 0.9347659037268785},{'precision': 0.7649164677804295},0.518923
2,0.085000,0.239274,{'accuracy': 0.9400076842856359},{'precision': 0.7799688635184224},0.578968


[I 2025-02-22 00:37:18,058] Trial 0 finished with value: 0.23927371203899384 and parameters: {'learning_rate': 0.0003, 'batch_size': 64}. Best is trial 0 with value: 0.23927371203899384.


...

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-1.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/home/bohan-jiang/start/232-stat-nlp/232-cdecr/src/trainer.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Acc,Precision,F1
1,0.101400,0.202302,{'accuracy': 0.9385806026675448},{'precision': 0.7385044124477473},0.586932
2,0.085600,0.248294,{'accuracy': 0.9391294802129645},{'precision': 0.7650632911392405},0.576718


[I 2025-02-22 05:05:27,761] Trial 15 finished with value: 0.20230181515216827 and parameters: {'learning_rate': 0.0003, 'batch_size': 64}. Best is trial 2 with value: 0.20230181515216827.


Best hyperparameters: {'learning_rate': 0.0003, 'batch_size': 64}
